# Imports, Ollama Download, and Environment Config

In [1]:
import os, fitz, warnings, subprocess, time
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [2]:
!curl https://ollama.ai/install.sh | sh
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", category=FutureWarning)

OLLAMA_MODEL='gemma:7b'
# Set it at the OS level
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10091    0 10091    0     0  37562      0 --:--:-- --:--:-- --:--:-- 37652
######################################################################## 100.0%##O#-#                                                                        #                                                    31.8%
>>> Installing ollama to /usr/local/bin...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


# Document Loading

In [3]:
# Function to extract text from a PDF file
def load_pdf(file_path):
    # Open the PDF file
    document = fitz.open(file_path)
    text = ''
    # Iterate through each page in the PDF
    for page in document:
        # Extract text from each page and concatenate it
        text += page.get_text()
    document.close()  # Close the document
    return text

# Path to your PDF file
pdf_path = 'sp287.pdf'
# Load the text from the PDF
data = load_pdf(pdf_path)

# Splitting

In [4]:
# Adjust chunk_size and chunk_overlap based on specific needs
chunk_size = 2000
chunk_overlap = 50

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
all_splits = text_splitter.split_text(data)

# Index

In [5]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
class SimpleDocument:
    def __init__(self, content):
        self.page_content = content
        self.metadata = {}

# Convert your list of strings to list of SimpleDocument
document_objects = [SimpleDocument(content) for content in all_splits]

vectorstore = Chroma.from_documents(
    documents=document_objects,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

In [7]:
question = "What were the causes of anomalies in the Apollo missions?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [8]:
docs[0]

Document(page_content='the same: The failure had to be understood and, if applicable, some\ncorrective action taken. This might involve design change,\nreinspection, or perhaps procedural change.\nI will conﬁne my remarks to anomalies that occurred during the\nﬁrst ﬁve manned Apollo ﬂights. The number of anomalies for each\nmission are given in table 1-III. Note that, even though each of the\nﬂights was completely successful and met all its objectives, the\nnumber of anomalies went quite high. Perhaps this is the best proof\nof the validity of the Apollo design concept: The spacecraft were\ndesigned for mission success.\n\xa0\nSpacecraft\nNumber of anomalies\nCommand and service\nmodule\nLunar module\n.\nApollo 7\n22\n-\nApollo 8\n8\n-\nApollo 9\n14\n12\nApollo 10\n23\n15\nApollo 11\n9\n13\nLet us look at just one example. On Apollo 10, during several of the\nlunar orbits, a critical fuel-cell temperature started to oscillate\nsigniﬁcantly, as shown in ﬁgure 1-5. Normally, this tempera

# Model

# Set up Ollama Server

In [9]:
# Start ollama as a backrgound process
command = "nohup ollama serve&"

# Use subprocess.Popen to start the process in the background
process = subprocess.Popen(command,
                            shell=True,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)
print("Process ID:", process.pid)
# Could also point elsewhere such as: !OLLAMA_HOST=https://ollama-demo.fly.dev:443
time.sleep(5)  # Makes Python wait for 5 seconds

Process ID: 297


In [10]:
!ollama -v

ollama version is 0.1.37


In [11]:
!ollama pull gemma:7b > /dev/null 2>&1

# RAG Chain

In [12]:
# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "gemma:7b"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [13]:
# Question
chain.invoke("What were the key design principles emphasized in the Apollo spacecraft?")

'**Key design principles emphasized in the Apollo spacecraft:**\n\n- Simplify operations.\n- Minimize interfaces.\n- Make maximum use of experience gained from previous manned-space programs.\n- Utilize established technology for areas where performance and reliability goals had already been realized.\n- Prevent the need for new hardware design by exploiting existing solutions.\n- Establish clear development requirements with suitable backup capabilities.\n- Prioritize safety by selecting appropriate design features, qualified components, and reliable operating principles.'

### Test for document text extraction, change from RawNB to Code if you need it